In [33]:
# 允许多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
# 设置图像大小
from IPython.core.pylabtools import figsize
figsize(20,20)
import datetime

In [44]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from pickle import load

In [4]:
# 导入数据
path = './csvs/pms_dfqp_lda_7days.csv'
columns = [
'fuid',  
'fplatformname',  
'label_score', 
'lifespan', 
'signup_to_now_days', 
'total_silver_coin', 
'total_gold_bar',  
'carrying_silver_coin',  
'carrying_gold_bar',  
'safebox_silver_coin',  
'safebox_gold_bar',  
'if_vip', 
'play_duration_life', 
'play_duration_30day', 
'play_duration_7day', 
'play_innings_life', 
'play_innings_30day', 
'play_innings_7day', 
'win_innings_life', 
'win_innings_7day', 
'lose_innings_life', 
'lose_innings_7day', 
'win_rate_life', 
'win_rate_7day', 
'pay_count_life', 
'pay_count_30day', 
'pay_count_7day', 
'pay_sum_life', 
'pay_sum_30day', 
'pay_sum_7day', 
'login_count_life', 
'login_count_7day', 
'bankrupt_count_life', 
'bankrupt_count_7day', 
'relieve_count_life', 
'relieve_silver_coins_life',  
'match_innings_life', 
'match_duration_life', 
'match_win_rate_life',
'recent_login_series_days', 
'latest_play_coins_win',
'latest_play_coins_lose',  
'last_active_to_now_days', 
'last_pay_to_now_days', 
'if_away', 
'dt',
]

data = pd.read_csv(path,names=columns,skiprows=1)
data.head(10)

,fuid,fplatformname,label_score,lifespan,signup_to_now_days,total_silver_coin,total_gold_bar,carrying_silver_coin,carrying_gold_bar,safebox_silver_coin,...,match_innings_life,match_duration_life,match_win_rate_life,recent_login_series_days,latest_play_coins_win,latest_play_coins_lose,last_active_to_now_days,last_pay_to_now_days,if_away,dt
0,19643563,云南,8,215,218,0,31,0,31,0,...,68,5298,0.6324,0,0,3520,3,174,0,2018-07-02
1,21140588,云南,9,2,2,60175,0,2199,0,57976,...,0,0,1.0000,2,0,2400,0,0,0,2018-07-02
2,20575943,云南,8,108,110,259237,10,259237,10,0,...,8,570,0.3750,0,0,36000,2,38,0,2018-07-02
3,20794575,云南,8,76,76,5932,86,5932,86,0,...,145,11465,0.4552,2,0,2800,0,0,0,2018-07-02
4,10166206,云南,9,465,469,0,0,0,0,0,...,1275,33265,0.0000,0,0,13460,4,0,0,2018-07-02
5,20517072,云南,5,115,117,2540,0,2540,0,0,...,0,0,1.0000,0,0,1300,2,15,0,2018-07-02
6,20858932,云南,9,64,64,30120,0,30120,0,0,...,0,0,1.0000,1,3000,0,0,0,0,2018-07-02
7,21084628,云南,8,10,15,2460,0,2460,0,0,...,0,0,1.0000,0,0,6000,5,0,1,2018-07-02
8,7089045,云南,8,520,521,7043,0,7043,0,0,...,11326,953247,0.0004,0,0,540,1,355,0,2018-07-02
9,19888253,云南,9,192,192,0,0,0,0,0,...,0,0,1.0000,1,0,14353,0,0,0,2018-07-02


In [6]:
# 从关系矩阵图中看出，有不少变量之间的相关性较高，我们通过卡方检验，筛选出10个对结果影响最大的数据特征
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE 
from scipy.stats import pearsonr
# 将数据分为输入数据和输出结果
array= data.values
X = array[:,2:44].astype(np.float32)
y = array[:,44].astype(np.float32)
# 调整数据尺度
transformer = MinMaxScaler(feature_range=(0,1))
newX = transformer.fit_transform(X)
# 特征选定
test = SelectKBest(score_func=chi2,k=10)
fit = test.fit(X,y)
set_printoptions(precision=3)
#fit.transform(X)
a = Series(fit.scores_)
b = data.iloc[:,2:44].columns
c = Series(b,a)
print("没做01变换的")
print(c.sort_index(ascending=False))

fit2 = test.fit(newX,y)
a1 = Series(fit2.scores_)
b1 = data.iloc[:,2:44].columns
c1 = Series(b1,a1)
print("做了01变换的")
print(c1.sort_index(ascending=False))

没做01变换的
4.693374e+08           play_duration_life
3.315888e+08    relieve_silver_coins_life
1.825247e+08          match_duration_life
1.572758e+08          safebox_silver_coin
1.194264e+08            total_silver_coin
7.562695e+07          play_duration_30day
2.087542e+07           play_duration_7day
4.879964e+06            play_innings_life
2.799971e+06            lose_innings_life
2.466078e+06         carrying_silver_coin
1.952583e+06           match_innings_life
1.848214e+06             win_innings_life
7.685370e+05           play_innings_30day
4.030352e+05             login_count_life
2.378433e+05       latest_play_coins_lose
2.086795e+05            play_innings_7day
1.168749e+05            lose_innings_7day
8.040773e+04             win_innings_7day
7.494414e+04          bankrupt_count_life
4.144878e+04           relieve_count_life
2.240524e+04     recent_login_series_days
1.785749e+04             login_count_7day
1.504395e+04                 pay_sum_life
1.046002e+04            ca

In [7]:
validation_size = 0.2
seed = 7
X_t,X_v,y_t,y_v = train_test_split(X,y,test_size=validation_size,random_state=seed)
# 通过分类准确度来评估算法
num_folds = 10
scoring = 'accuracy'

## 算法
models = {}
# 逻辑回归
models['LR'] = LogisticRegression()
# 线性判别分析
models['LDA'] = LinearDiscriminantAnalysis()
# K近邻
models['KNN'] = KNeighborsClassifier()
# 分类与回归树
models['CART'] = DecisionTreeClassifier()
# 贝叶斯分类器
models['NB'] = GaussianNB()
# 支持向量机
models['SVM'] = SVC()

results = []
for key in models:
    kfold = KFold(n_splits=num_folds,random_state=seed)
    cv_results = cross_val_score(models[key],X_t,y_t,cv=kfold,scoring=scoring)
    results.append(cv_results)
    print('%s: %f (%f)'% (key,cv_results.mean(),cv_results.std()))

# 可能由于数据分布的不均导致算法不够准确
# 于是将数据进行正态化处理
pipelines = {}
pipelines['ScalerLR'] = Pipeline([('Scaler',StandardScaler()),('LR',LogisticRegression())])
pipelines['ScalerLDA'] = Pipeline([('Scaler',StandardScaler()),('LDA',LinearDiscriminantAnalysis())])
pipelines['ScalerKNN'] = Pipeline([('Scaler',StandardScaler()),('KNN',KNeighborsClassifier())])
pipelines['ScalerCART'] = Pipeline([('Scaler',StandardScaler()),('CART',DecisionTreeClassifier())])
#pipelines['ScalerNB'] = Pipeline([('Scaler',StandardScaler()),('NB',GaussianNB)])
pipelines['ScalerSVM'] = Pipeline([('Scaler',StandardScaler()),('SVM',SVC())])

for key in pipelines:
    kfold = KFold(n_splits=num_folds,random_state=seed)
    cv_results = cross_val_score(pipelines[key],X_t,y_t,cv=kfold,scoring=scoring)
    results.append(cv_results)
    print('%s: %f (%f)'%(key,cv_results.mean(),cv_results.std()))

# 集成算法
ensembles = {}
ensembles['ScaledAB'] = Pipeline([('Scaler',StandardScaler()),('AB',AdaBoostClassifier())])
ensembles['ScaledGBM'] = Pipeline([('Scaler',StandardScaler()),('GBM',GradientBoostingClassifier())])
ensembles['ScaledRF'] = Pipeline([('Scaler',StandardScaler()),('RFR',RandomForestClassifier())])
ensembles['ScaledET'] = Pipeline([('Scaler',StandardScaler()),('ETR',ExtraTreesClassifier())])

for key in ensembles:
    kfold = KFold(n_splits=num_folds,random_state=seed)
    cv_result = cross_val_score(ensembles[key],X_t,y_t,cv=kfold,scoring=scoring)
    results.append(cv_result)
    print('%s: %f (%f)'% (key,cv_result.mean(),cv_result.std()))

KNN: 0.751250 (0.019138)


c:\python27\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LDA: 0.798750 (0.013405)
SVM: 0.765250 (0.017861)
NB: 0.544750 (0.022007)
CART: 0.748000 (0.015033)
LR: 0.781625 (0.017167)
ScalerLDA: 0.798750 (0.013405)
ScalerCART: 0.749625 (0.015663)
ScalerKNN: 0.783125 (0.011072)
ScalerLR: 0.807750 (0.010923)
ScalerSVM: 0.805250 (0.012013)
ScaledET: 0.789875 (0.019247)
ScaledAB: 0.809750 (0.010866)
ScaledRF: 0.796375 (0.010584)
ScaledGBM: 0.813250 (0.014374)


In [47]:
model = GradientBoostingClassifier()
model.fit(X_t,y_t)
from pickle import dump
save_model = './csvs/GBM.sav'
with open(save_model,'wb') as model_f:
    dump(model,model_f)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [14]:
import pandas as pd
from pandas import DataFrame
from pickle import load
import datetime

# 获取昨天的日期
now = datetime.datetime.now()-datetime.timedelta(days = 1)
date = now.strftime('%Y%m%d')
# 拼接成文件名
predict_filename = 'training_data_yunnan_'+date+'.csv'

# 获取预测文件
predict_dir = '../data/dfqp/'
predict_path = predict_dir+predict_filename

# 读取文件
predict_file = pd.read_csv(predict_path)
# 获取预测字段
predict_values = predict_file.values
predict_X = predict_values[:,2:44]

# 获取模型
model_path = '../models/dfqp/GBM.sav'
# 利用模型进行预测
with open(model_path,'rb') as model_f:
    loaded_model = load(model_f)
    predict_y = loaded_model.predict(predict_X)

# 生成结果文件
fuid = predict_file['fuid']
fplatformname = predict_file['fplatformname']
dt = predict_file['dt']
result = pd.DataFrame({'dt':dt,'fplatformname':fplatformname,'fuid':fuid,'predict':predict_y})

# 保存结果文件
save_filename = 'predict_data_yunnan_'+date+'.csv'
predict_save_path = predict_dir+save_filename
result.to_csv(predict_save_path)

In [46]:
from pandas import DataFrame
from pickle import load
import datetime
predict_file = pd.read_csv('./csvs/pms_dfqp_GBM_7days.csv')
predict_values = predict_file.values
predict_X = predict_values[:,2:35]

model_path = './csvs/liushi_predict_dfqp_GBM.sav'
with open(model_path,'rb') as model_f:
    loaded_model = load(model_f)
    predict_y = loaded_model.predict(predict_X)

fuid = predict_file['fuid']
fplatformname = predict_file['fplatformname']
dt = predict_file['dt']
result = pd.DataFrame({'dt':dt,'fplatformname':fplatformname,'fuid':fuid,'predict':predict_y})

save_filename = 'pmr_dfqp_GBM_7days_'+date+'.csv'
predict_save_path = './csvs/'+save_filename
result.to_csv(predict_save_path,index=False)
